In [5]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://www.flipkart.com/search?q={0}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={1}'
#{0}:format後的第一個位置、{1}:第二個


def get_urls(url, query, start_page, end_page): #建立可自行輸入"查詢值"和"start~end_page"的url清單
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    return urls

def get_parse(url):
    #headers = {"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Mobile Safari/537.36FKUA/msite/0.0.3/msite/Mobile"}
    #res = requests.get(url, headers=headers)
    res = requests.get(url)
    if res.status_code == requests.codes.ok:
        print("Request Succeeded")
        return BeautifulSoup(res.text, "lxml") #常用的解析器有三種："lxml","html5lib","html.parser"，官方建議使用解析速度較快的"lxml"
    else:
        return print("Request Failed")
    
def get_goods(soup):
    goods = []
    rows = soup.find("div", attrs={"class":"_1HmYoV _35HD7C", "style":"flex-grow:1;overflow:auto"}).find_all("div", "_3liAhj")
    #rows = soup.selct("._3liAhj")
    for row in rows:
        try:
            name = row.find("a", attrs={"class":"_2cLu-l"}).text
            #name = row.select("._2cLu-l")[0].get_text()   #後面加[0]代表只找第一個，等同為select_one
            #name = row.find("div", class_="_3BTv9X").img["alt"]
        except:
            name = None
        try:
            #price = row.find("div", attrs={"class":"_1vC4OE"}).text
            price = row.select_one("._1vC4OE").get_text()
        except:
            price = None
        try:
            #price_2 = row.find("div", "_3auQ3N").text
            price_2 = row.select("._3auQ3N")[0].get_text()
        except:
            price_2 = None
        try:
            #discount = row.find("div", "VGWI6T").text
            discount = row.select_one(".VGWI6T span").get_text()
        except:
            discount = None
        try:
            #star = row.find("div", "hGSR34").text
            star = row.select(".hGSR34")[0].get_text()
        except:
            star = None
        try:
            #reviews = row.find("span", "_38sUEc").text
            reviews = row.select("._38sUEc")[0].get_text()
        except:
            reviews = None
            
        good = [name, price, price_2, discount, star, reviews]  #將爬取項目照此順序放入清單
        goods.append(good)  #將good清單加入goods清單內，會產生[[name, price, price_2, discount, star, reviews]]巢狀清單
    return goods

def scraping_bot(urls):
    all_goods = [["品名", "價格", "價格2", "折扣%", "評價", "評論數"]]  #巢狀清單(當作欄位名稱用)
    
    page = 1
    for url in urls:
        print("Scraping_Page" + str(page) + "...")
        page = page + 1
        soup = get_parse(url)
        goods = get_goods(soup)
        all_goods = all_goods + goods
        print("Wait 3 Seconds...")
        if soup.find("li", class_="a-disabled a-last"):  #每個網頁都不同，要自行改，但如果有限制end_page的話不寫這段也沒差
            break   #已經沒有下一頁
        time.sleep(3)
    return all_goods


def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)


#爬蟲主程式
if __name__ == "__main__":  #當被匯入時不執行的段落
#get_urls
    urls = get_urls(url, 'logitech', 1, 5)
    print(urls)
#get_data
    goods = scraping_bot(urls)
    df = pd.DataFrame(goods)     #用dataframe列出
    print(df)
    #for good in goods:          #用list列出
    #    print(good)
    save_to_csv(goods, "t.csv")

['https://www.flipkart.com/search?q=logitech&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1', 'https://www.flipkart.com/search?q=logitech&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=2', 'https://www.flipkart.com/search?q=logitech&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=3', 'https://www.flipkart.com/search?q=logitech&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=4', 'https://www.flipkart.com/search?q=logitech&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=5']
Scraping_Page1...
Request Succeeded
Wait 3 Seconds...
Scraping_Page2...
Request Succeeded
Wait 3 Seconds...
Scraping_Page3...
Request Succeeded
Wait 3 Seconds...
Scraping_Page4...
Request Succeeded
Wait 3 Seconds...
Scraping_Page5...
Request Succeeded
Wait 3 Seconds...
          0        1        2     3         4  \
0        品名       價格      價格2   折扣%        評價   
1     

In [40]:
get_parse("https://www.flipkart.com/search?q=logitech&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1")

Request Succeeded


<!DOCTYPE html>
<html lang="en">
<head>
<link href="https://rukminim1.flixcart.com" rel="dns-prefetch"/>
<link href="https://img1a.flixcart.com" rel="dns-prefetch"/>
<link href="//img1a.flixcart.com/www/linchpin/fk-cp-zion/css/app.chunk.0bf2d8.css" rel="stylesheet"/>
<link as="image" href="//img1a.flixcart.com/www/linchpin/fk-cp-zion/img/fk-logo_9fddff.png" rel="preload"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="102988293558" property="fb:page_id"/>
<meta content="658873552,624500995,100000233612389" property="fb:admins"/>
<meta content="noodp" name="robots"/>
<link href="https://img1a.flixcart.com/www/promos/new/20150528-140547-favicon-retina.ico" rel="shortcut icon"/>
<link href="/osdd.xml?v=2" rel="search" type="application/opensearchdescription+xml"/>
<meta content="website" property="og:type"/>
<meta content="Flipkart.com" name="og_site_name" property="og:site_name"/>
<!-- Appple Tou

In [39]:
df

,0,1,2,3,4,5
0,品名,價格,價格2,折扣%,評價,評論數
1,Logitech M100R Wired Optical Mouse,₹449,₹495,9% off,3.9,"(1,662)"
2,Logitech M90 Wired Optical Gaming Mouse,₹349,₹395,11% off,4.3,"(1,042)"
3,Logitech Mk235 Mouse & Wireless Laptop Keyboard,"₹1,299","₹1,995",34% off,4.3,"(1,675)"
4,Logitech MK240 Wireless Keyboard and Mouse Combo,"₹1,399","₹1,500",6% off,4.4,"(23,710)"
...,...,...,...,...,...,...
196,Logitech Dotionmo Mousepad,₹125,₹150,16% off,None,None
197,Logitech H570e Wired Headset,"₹4,900","₹4,995",1% off,None,None
198,Hermitshell Keyboard Case for Logitech,"₹3,205","₹4,167",23% off,None,None
199,"Logitech HD Webcam C525, Portable HD 720p Vide...","₹3,550","₹4,295",17% off,4.1,(14)


In [3]:
ds = []
d = ['name', 'price', 'price_2', 'discount', 'star, reviews']
ds.append(d)

In [4]:
ds

[['name', 'price', 'price_2', 'discount', 'star, reviews']]